In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm
from bert_model import BiLSTM_CRF_BERT
from utils import prepare, predict

from transformers import get_linear_schedule_with_warmup, AdamW

In [2]:
hparams = {
    'path':'/home/peitian_zhang/Data/NER/labeled_train.txt',
    'epochs': 20,
    'batch_size': 15,
    'embedding_dim': 768,
    'hidden_dim': 768,
    'device':'cuda:0',
    'bert':'ckiplab/albert-base-chinese',
}

In [3]:
tag2idx, vocab, loader = prepare(hparams)
hparams['vocab_size'] = len(vocab)
hparams['seq_length'] = loader.dataset.max_length

In [4]:
bert_model = BiLSTM_CRF_BERT(hparams,tag2idx).to(hparams['device'])

Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
total_steps = len(loader) * hparams['epochs']

optimizer = optim.AdamW(bert_model.parameters(),lr=0.001)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

for epoch in range(hparams['epochs']):
    tqdm_ = tqdm(enumerate(loader))
    total_loss = 0

    for step,x in tqdm_:
        bert_model.zero_grad()
        loss = bert_model.neg_log_likelihood(x)
        loss.backward()

        # prevent gradient explosion
        # torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

        optimizer.step()
        # scheduler.step()

        total_loss += loss.item()
        tqdm_.set_description("epoch {:d} , step {:d} , loss: {:.4f}".format(epoch+1, step, total_loss/(step+1)))

epoch 1 , step 20 , loss: -1627.4039: : 21it [00:12,  1.74it/s]

In [ ]:
with torch.no_grad():
    record = next(iter(loader))
    _, tag_seq = bert_model(record['token'], record['attn_mask'])
    print("Prediction:{}\n Ground Truth:{}".format(tag_seq, record['label']))

In [ ]:
from transformers import AutoModel,BertTokenizerFast, BertTokenizer,BertModel

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
encoded = tokenizer.encode_plus('窦志成获奖', pad_to_max_length=True, max_length=10, truncation=True, return_tensors='pt')
encoded['input_ids'], encoded['attention_mask']

In [34]:
model = AutoModel.from_pretrained('ckiplab/albert-base-chinese-ner',num_labels=22)
# a = model(encoded['input_ids'], encoded['attention_mask'], labels=torch.tensor([[14,15,15,3,3,2,2,2,2,2]]))
model(encoded['input_ids'], encoded['attention_mask'])

Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese-ner and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.1606,  0.2864, -0.1147,  ..., -0.0842,  0.1854,  0.0919],
         [ 1.6543,  0.3195, -0.3530,  ...,  0.0622, -0.7103, -0.3791],
         [ 0.0810, -0.5529, -0.0126,  ..., -0.1332,  0.7997, -1.2569],
         ...,
         [-0.7667, -0.0724, -0.3125,  ..., -0.4537,  0.4974, -0.7879],
         [-0.7574, -0.0837, -0.3519,  ..., -0.4553,  0.5126, -0.7774],
         [-0.7646, -0.0973, -0.4095,  ..., -0.4482,  0.5277, -0.7301]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 6.7524e-02,  4.7782e-02,  1.1634e-01, -1.1055e-02,  7.7158e-02,
         -1.3929e-01,  6.3112e-02,  1.6692e-01, -9.6543e-02, -4.2006e-02,
         -6.1763e-02, -8.4560e-02, -2.0876e-02, -5.5645e-02, -3.1521e-01,
         -4.2518e-02, -7.5884e-02, -3.6048e-02, -8.2107e-03,  8.8506e-02,
          3.7791e-02, -1.4655e-01, -3.5113e-02,  8.0328e-02, -7.6507e-02,
         -3.3539e-02,  2.8168e-01,  2.8507e-01,  5.8243e-02, -5.4598e-02,
         -6.

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel,AutoModel

START_TAG = '<START>'
STOP_TAG = '<END>'
PAD_TAG = '<PAD>'

class BERT_NER(nn.Module):
    """
        fine-tune pretrained BERT-NER model on our dataset
    """

    def __init__(self, hparams, tag2idx):
        super().__init__()

        self.batch_size = hparams['batch_size']
        self.seq_length = hparams['seq_length']

        self.device = hparams['device']

        self.tag2idx = tag2idx
        self.tagset_size = len(tag2idx)
        self.idx2tag = {v:k for k,v in tag2idx.items()}

        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim // 2, bidirectional=True)

        self.bert = AutoModel.from_pretrained(
            'ckiplab/albert-base-chinese-ner',
            # output hidden embedding of each transformer layer
        )

        self.hidden2tag = nn.Linear(self.embedding_dim, self.tagset_size)
        self.Softmax = nn.Softmax(dim=-1)
    
    def _bert_encode(token, attn_mask):
        output = self.bert(token, token_mask)
        embedding = output['last_hidden_state']
        tag_prob = self.Softmax(self.hidden2tag(embedding))

    
    def forward(x):


In [39]:
model = AutoModel.from_pretrained('ckiplab/albert-base-chinese', output_hidden_states=True)
# a = model(encoded['input_ids'], encoded['attention_mask'], labels=torch.tensor([[14,15,15,3,3,2,2,2,2,2]]))
a = model(encoded['input_ids'], encoded['attention_mask'])

Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
a['hidden_states']

13